<a href="https://colab.research.google.com/github/imymemineyay/Mini-Project-3/blob/main/1%EC%B0%A8%EC%A0%84%EC%B2%98%EB%A6%AC_VIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 사용할 라이브러리 로딩

In [ ]:
import numpy as np # Numpy
import pandas as pd # Pandas
import matplotlib as mpl #Matplotlib 세팅용
import matplotlib.pyplot as plt # 시각화 도구
import seaborn as sns # 시각화 도구
from sklearn.model_selection import train_test_split # 데이터셋 분리
from sklearn.cluster import KMeans # 클러스터링
from sklearn.metrics import silhouette_score # 실루엣 점수
import xgboost as xgb # XGBoost
from sklearn.model_selection import GridSearchCV # 그리드 서치
from sklearn.metrics import accuracy_score, precision_score # 평가 지표
from sklearn.metrics import recall_score, confusion_matrix, roc_auc_score, f1_score # 평가 지표


import warnings # 경고문 제거용


%matplotlib inline
%config Inlinebackend.figure_format = 'retina'

# 한글 폰트 설정
mpl.rc('font', family='D2Coding')
# 유니코드에서 음수 부호 설정
mpl.rc('axes', unicode_minus = False)

warnings.filterwarnings('ignore')
sns.set(font="D2Coding", rc={"axes.unicode_minus":False}, style='darkgrid')
plt.rc('figure', figsize=(10,8))

# 데이터 로딩

In [ ]:
data = pd.read_excel('C:\\k_digital\\source\\sparta_data\\train_test_origin.xlsx')

In [ ]:
data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin1,Cabin2,Combi,Cabin3,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,0.0,B,0.0,B0,P,B/0/P,TRAPPIST-1e,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0.0
1,0002_01,Earth,0.0,F,0.0,F0,S,F/0/S,TRAPPIST-1e,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1.0
2,0003_01,Europa,0.0,A,0.0,A0,S,A/0/S,TRAPPIST-1e,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0.0
3,0003_02,Europa,0.0,A,0.0,A0,S,A/0/S,TRAPPIST-1e,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0.0
4,0004_01,Earth,0.0,F,1.0,F1,S,F/1/S,TRAPPIST-1e,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1.0


In [ ]:
data.shape

(12970, 18)

In [ ]:
data.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin1', 'Cabin2', 'Combi',
       'Cabin3', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService',
       'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Transported'],
      dtype='object')

In [ ]:
data.VIP.value_counts()

0.0    12401
1.0      273
Name: VIP, dtype: int64

In [ ]:
data.isna().sum()

PassengerId        0
HomePlanet       279
CryoSleep        310
Cabin1           166
Cabin2           166
Combi            166
Cabin3           166
Cabin            166
Destination      266
Age              270
VIP              296
RoomService      194
FoodCourt        218
ShoppingMall     210
Spa              216
VRDeck           204
Name             294
Transported     4277
dtype: int64

# 필요없다고 판단된 피처 삭제
- 'PassengerId','Combi','Cabin','Name','Transported'

In [ ]:
data.drop(['PassengerId','Combi','Cabin','Name','Transported'],axis=1,inplace=True)

In [ ]:
data.columns

Index(['HomePlanet', 'CryoSleep', 'Cabin1', 'Cabin2', 'Cabin3', 'Destination',
       'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa',
       'VRDeck'],
      dtype='object')

In [ ]:
data.head()

,HomePlanet,CryoSleep,Cabin1,Cabin2,Cabin3,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,Europa,0.0,B,0.0,P,TRAPPIST-1e,39.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Earth,0.0,F,0.0,S,TRAPPIST-1e,24.0,0.0,109.0,9.0,25.0,549.0,44.0
2,Europa,0.0,A,0.0,S,TRAPPIST-1e,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0
3,Europa,0.0,A,0.0,S,TRAPPIST-1e,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0
4,Earth,0.0,F,1.0,S,TRAPPIST-1e,16.0,0.0,303.0,70.0,151.0,565.0,2.0


# EDA NaN값 판단

In [ ]:
data.isna().sum()

HomePlanet      279
CryoSleep       310
Cabin1          166
Cabin2          166
Cabin3          166
Destination     266
Age             270
VIP             296
RoomService     194
FoodCourt       218
ShoppingMall    210
Spa             216
VRDeck          204
dtype: int64

# 전처리

## 'True','False'를 가지고 있는 Object 타입을 Boolean 타입으로 변경

In [ ]:
data.Cabin3.value_counts()

S    6450
P    6354
Name: Cabin3, dtype: int64

In [ ]:
# Cabin3의 P(좌현)를 False으로 S(우현)를 True로 변경
data['Cabin3'].replace({'P': True,'S': False}, inplace=True)

# boolean으로 변환
data['CryoSleep'] = data['CryoSleep'].astype(bool)
data['VIP'] = data['VIP'].astype(bool)
data['Cabin3'] = data['Cabin3'].astype(bool)

# 변환 확인
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    12691 non-null  object 
 1   CryoSleep     12970 non-null  bool   
 2   Cabin1        12804 non-null  object 
 3   Cabin2        12804 non-null  float64
 4   Cabin3        12970 non-null  bool   
 5   Destination   12704 non-null  object 
 6   Age           12700 non-null  float64
 7   VIP           12970 non-null  bool   
 8   RoomService   12776 non-null  float64
 9   FoodCourt     12752 non-null  float64
 10  ShoppingMall  12760 non-null  float64
 11  Spa           12754 non-null  float64
 12  VRDeck        12766 non-null  float64
dtypes: bool(3), float64(7), object(3)
memory usage: 1.0+ MB


In [ ]:
data.Cabin3.value_counts()

True     6520
False    6450
Name: Cabin3, dtype: int64

## 나머지 object타입 인코딩화

In [ ]:
# object 타입의 데이터들 더미화
train_encoding = pd.get_dummies(data['HomePlanet'])
data=data.drop('HomePlanet',axis=1)
data = data.join(train_encoding)

train_encoding = pd.get_dummies(data['Cabin1'])
# 기존의 팀명 컬러 삭제
data=data.drop('Cabin1',axis=1)
data = data.join(train_encoding)


train_encoding = pd.get_dummies(data['Destination'])
# 기존의 팀명 컬러 삭제
data=data.drop('Destination',axis=1)
data = data.join(train_encoding)

In [ ]:
data.head()

,CryoSleep,Cabin2,Cabin3,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,B,C,D,E,F,G,T,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
0,False,0.0,True,39.0,False,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,1
1,False,0.0,False,24.0,False,109.0,9.0,25.0,549.0,44.0,...,0,0,0,0,1,0,0,0,0,1
2,False,0.0,False,58.0,True,43.0,3576.0,0.0,6715.0,49.0,...,0,0,0,0,0,0,0,0,0,1
3,False,0.0,False,33.0,False,0.0,1283.0,371.0,3329.0,193.0,...,0,0,0,0,0,0,0,0,0,1
4,False,1.0,False,16.0,False,303.0,70.0,151.0,565.0,2.0,...,0,0,0,0,1,0,0,0,0,1


## 피처 스케일링

In [ ]:
# 스케일링을 위한 함수 생성
col = ['Cabin2', 'Age', 'RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
def data_scaled(df, col):
    for i in col:
        data_mean = df[i].mean()
        data_std = df[i].std()
        scaled = (df[i]-data_mean)/data_std
        df[i]=scaled
    return df

data_scaled(data, col)

,CryoSleep,Cabin2,Cabin3,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,B,C,D,E,F,G,T,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
0,False,-1.174864,True,0.710894,False,-0.343152,-0.284403,-0.294957,-0.272138,-0.259273,...,1,0,0,0,0,0,0,0,0,1
1,False,-1.174864,False,-0.331770,False,-0.174435,-0.278708,-0.252478,0.214786,-0.221901,...,0,0,0,0,1,0,0,0,0,1
2,False,-1.174864,False,2.031602,True,-0.276594,1.978442,-0.294957,5.683589,-0.217654,...,0,0,0,0,0,0,0,0,0,1
3,False,-1.174864,False,0.293829,False,-0.343152,0.527462,0.335431,2.680448,-0.095344,...,0,0,0,0,0,0,0,0,0,1
4,False,-1.172914,False,-0.887857,False,0.125850,-0.240108,-0.038384,0.228976,-0.257575,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12965,True,1.741991,False,0.363340,False,-0.343152,-0.284403,-0.294957,-0.272138,-0.259273,...,0,0,0,0,0,1,0,0,0,1
12966,False,NaN,True,0.919427,False,-0.343152,0.251567,-0.266071,-0.263269,-0.136963,...,0,0,0,0,0,0,0,0,0,1
12967,True,-0.597732,True,NaN,False,-0.343152,-0.284403,-0.294957,-0.272138,-0.259273,...,0,0,1,0,0,0,0,1,0,0
12968,False,-0.595783,True,NaN,False,-0.343152,1.411465,-0.294957,-0.272138,0.184950,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
data.head()

,CryoSleep,Cabin2,Cabin3,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,B,C,D,E,F,G,T,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
0,False,-1.174864,True,0.710894,False,-0.343152,-0.284403,-0.294957,-0.272138,-0.259273,...,1,0,0,0,0,0,0,0,0,1
1,False,-1.174864,False,-0.331770,False,-0.174435,-0.278708,-0.252478,0.214786,-0.221901,...,0,0,0,0,1,0,0,0,0,1
2,False,-1.174864,False,2.031602,True,-0.276594,1.978442,-0.294957,5.683589,-0.217654,...,0,0,0,0,0,0,0,0,0,1
3,False,-1.174864,False,0.293829,False,-0.343152,0.527462,0.335431,2.680448,-0.095344,...,0,0,0,0,0,0,0,0,0,1
4,False,-1.172914,False,-0.887857,False,0.125850,-0.240108,-0.038384,0.228976,-0.257575,...,0,0,0,0,1,0,0,0,0,1


In [ ]:
data1 = data.dropna(axis=0)

In [ ]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11553 entries, 0 to 12969
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CryoSleep      11553 non-null  bool   
 1   Cabin2         11553 non-null  float64
 2   Cabin3         11553 non-null  bool   
 3   Age            11553 non-null  float64
 4   VIP            11553 non-null  bool   
 5   RoomService    11553 non-null  float64
 6   FoodCourt      11553 non-null  float64
 7   ShoppingMall   11553 non-null  float64
 8   Spa            11553 non-null  float64
 9   VRDeck         11553 non-null  float64
 10  Earth          11553 non-null  uint8  
 11  Europa         11553 non-null  uint8  
 12  Mars           11553 non-null  uint8  
 13  A              11553 non-null  uint8  
 14  B              11553 non-null  uint8  
 15  C              11553 non-null  uint8  
 16  D              11553 non-null  uint8  
 17  E              11553 non-null  uint8  
 18  F     

In [ ]:
X = data1.drop(['VIP'],axis=1)

In [ ]:
y = data1['VIP'].replace({True:1,False:0})

In [ ]:
X.head()

,CryoSleep,Cabin2,Cabin3,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Earth,...,B,C,D,E,F,G,T,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
0,False,-1.174864,True,0.710894,-0.343152,-0.284403,-0.294957,-0.272138,-0.259273,0,...,1,0,0,0,0,0,0,0,0,1
1,False,-1.174864,False,-0.331770,-0.174435,-0.278708,-0.252478,0.214786,-0.221901,1,...,0,0,0,0,1,0,0,0,0,1
2,False,-1.174864,False,2.031602,-0.276594,1.978442,-0.294957,5.683589,-0.217654,0,...,0,0,0,0,0,0,0,0,0,1
3,False,-1.174864,False,0.293829,-0.343152,0.527462,0.335431,2.680448,-0.095344,0,...,0,0,0,0,0,0,0,0,0,1
4,False,-1.172914,False,-0.887857,0.125850,-0.240108,-0.038384,0.228976,-0.257575,1,...,0,0,0,0,1,0,0,0,0,1


In [ ]:
y.head()

0    0
1    0
2    1
3    0
4    0
Name: VIP, dtype: int64

In [ ]:
y.value_counts()

0    11044
1      509
Name: VIP, dtype: int64

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X, y, stratify = y, random_state=109)

In [ ]:
import xgboost as xgb

model = xgb.XGBClassifier(n_estimators = 500, max_depth = 5, random_state = 109, use_missing=False)
model.fit(X_train,y_train)
pred = model.predict(X_test)
pred

[14:23:18] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "use_missing" } are not used.



array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test, pred) 

0.9501557632398754

In [ ]:
print(confusion_matrix(y_test,pred))

[[2743   19]
 [ 125    2]]


In [ ]:
f1_score(y_test,pred)

0.027027027027027025

In [ ]:
# xgbo = xgb.XGBClassifier()

# params = {
#     'max_depth':[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, None],
#     'learning_rate':[0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4],
#     'gamma':[0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5],
#     'random_state':[109]
# }

# gs = GridSearchCV(xgbo, param_grid = params, cv = 3, refit = True, n_jobs=-1)
# gs.fit(X_train, y_train)

In [ ]:
print(confusion_matrix(y_test,pred))

[[2743   19]
 [ 125    2]]


# 오버샘플링

In [ ]:
! pip install -U imbalanced-learn

In [ ]:
from imblearn.over_sampling import *

In [ ]:
X_samp, y_samp = RandomOverSampler(random_state=109).fit_resample(X,y)

In [ ]:
print(X_samp.shape)
print(y_samp.shape)

(22088, 23)
(22088,)


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_samp, y_samp, stratify = y_samp, random_state=109)

In [ ]:
model = xgb.XGBClassifier(n_estimators = 500, max_depth = 5, random_state = 109, use_missing=False)
model.fit(X_train,y_train)
pred = model.predict(X_test)
pred

[14:23:43] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "use_missing" } are not used.



array([0, 1, 1, ..., 0, 1, 1])

In [ ]:
accuracy_score(y_test, pred) 

0.9833393697935531

In [ ]:
print(confusion_matrix(y_test,pred))

[[2669   92]
 [   0 2761]]


In [ ]:
f1_score(y_test,pred)

0.983612397577485

In [ ]:
! pip install hyperopt

In [ ]:
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error

# hyperopt 적용을 위한 설정 값 담기
xgb_search_space = {'max_depth': hp.quniform('max_depth', 5, 20, 1),
                    'min_child_weight': hp.quniform('min_child_weight', 1, 2, 1),
                    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
                    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1)
               }

In [ ]:
# 실행을 위한 함수 정의
from xgboost import XGBClassifier
def objective_func(search_space):
    xgb_clf = XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth']),
                            min_child_weight=int(search_space['min_child_weight']),
                            learning_rate=search_space['learning_rate'],
                            colsample_bytree=search_space['colsample_bytree'], 
                            eval_metric='logloss')
    accuracy = cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3)
    return {'loss':-1 * np.mean(accuracy), 'status': STATUS_OK}

In [ ]:
# 최적의 파라미터 찾기
from sklearn.model_selection import cross_val_score
trial_val = Trials()
best = fmin(fn=objective_func,
            space=xgb_search_space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trial_val, rstate=np.random.default_rng(seed=9))
print('best:', best)

100%|███████████████████████████████████████████████| 50/50 [01:56<00:00,  2.33s/trial, best loss: -0.9852710370638658]
best: {'colsample_bytree': 0.5992654290397839, 'learning_rate': 0.13626925126739006, 'max_depth': 20.0, 'min_child_weight': 1.0}


In [ ]:
# 모델링
xgb_model = XGBClassifier(n_estimators=200, learning_rate=round(best['learning_rate'], 5), 
                            max_depth=int(best['max_depth']), min_child_weight=int(best['min_child_weight']),
                            colsample_bytree=round(best['colsample_bytree'], 5)
                           )
xgb_model.fit(X_train, y_train)
pred= xgb_model.predict(X_test)
print('정확도 : ', accuracy_score(y_test, pred))
print('혼돈행렬 : \n', confusion_matrix(y_test, pred))

정확도 :  0.992212966316552
혼돈행렬 : 
 [[2718   43]
 [   0 2761]]


In [ ]:
pred= xgb_model.predict(X_test)
print('정확도 : ', accuracy_score(y_test, pred))
print('혼돈행렬 : \n', confusion_matrix(y_test, pred))

정확도 :  0.992212966316552
혼돈행렬 : 
 [[2718   43]
 [   0 2761]]


In [ ]:
pred= xgb_model.predict(X_train)
print('정확도 : ', accuracy_score(y_train, pred))
print('혼돈행렬 : \n', confusion_matrix(y_train, pred))

정확도 :  1.0
혼돈행렬 : 
 [[8283    0]
 [   0 8283]]


# VIP 결측치 채우기

In [ ]:
vip_data = pd.read_excel('C:\\k_digital\\source\\sparta_data\\na_vip.xlsx')

In [ ]:
vip_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin1,Cabin2,Combi,Cabin3,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0036_01,Earth,False,F,8.0,F8,S,F/8/S,55 Cancri e,15.0,NaN,0.0,492.0,48.0,20.0,401.0,Marina Leodger,0.0
1,0108_03,Earth,False,G,19.0,G19,S,G/19/S,TRAPPIST-1e,0.0,NaN,0.0,0.0,0.0,0.0,0.0,Oline Handertiz,1.0
2,0165_01,Mars,True,F,37.0,F37,P,F/37/P,TRAPPIST-1e,35.0,NaN,0.0,0.0,0.0,0.0,0.0,Graven Anche,1.0
3,0244_02,Mars,True,F,47.0,F47,S,F/47/S,55 Cancri e,14.0,NaN,0.0,0.0,0.0,0.0,0.0,Tous Sad,1.0
4,0593_01,Mars,False,D,24.0,D24,P,D/24/P,TRAPPIST-1e,NaN,NaN,43.0,152.0,182.0,1.0,2005.0,Hon Kra,0.0


In [ ]:
vip_data.drop(['PassengerId','Combi','Cabin','Name','Transported'],axis=1,inplace=True)

In [ ]:
vip_data.isna().sum()

HomePlanet        0
CryoSleep         0
Cabin1            9
Cabin2            9
Cabin3            9
Destination       7
Age               9
VIP             296
RoomService       5
FoodCourt         3
ShoppingMall      4
Spa               3
VRDeck            2
dtype: int64

In [ ]:
# Cabin3의 P(좌현)를 False으로 S(우현)를 True로 변경
vip_data['Cabin3'].replace({'P': True,'S': False}, inplace=True)

# boolean으로 변환
vip_data['CryoSleep'] = vip_data['CryoSleep'].astype(bool)

vip_data['Cabin3'] = vip_data['Cabin3'].astype(bool)

# 변환 확인
vip_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296 entries, 0 to 295
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    296 non-null    object 
 1   CryoSleep     296 non-null    bool   
 2   Cabin1        287 non-null    object 
 3   Cabin2        287 non-null    float64
 4   Cabin3        296 non-null    bool   
 5   Destination   289 non-null    object 
 6   Age           287 non-null    float64
 7   VIP           0 non-null      float64
 8   RoomService   291 non-null    float64
 9   FoodCourt     293 non-null    float64
 10  ShoppingMall  292 non-null    float64
 11  Spa           293 non-null    float64
 12  VRDeck        294 non-null    float64
dtypes: bool(2), float64(8), object(3)
memory usage: 26.1+ KB


In [ ]:
# object 타입의 데이터들 더미화
train_encoding = pd.get_dummies(vip_data['HomePlanet'])
vip_data=vip_data.drop('HomePlanet',axis=1)
vip_data = vip_data.join(train_encoding)

train_encoding = pd.get_dummies(vip_data['Cabin1'])
# 기존의 팀명 컬러 삭제
vip_data=vip_data.drop('Cabin1',axis=1)
vip_data = vip_data.join(train_encoding)


train_encoding = pd.get_dummies(vip_data['Destination'])
# 기존의 팀명 컬러 삭제
vip_data=vip_data.drop('Destination',axis=1)
vip_data = vip_data.join(train_encoding)

In [ ]:
# 스케일링을 위한 함수 생성
col = ['Cabin2', 'Age', 'RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
def data_scaled(df, col):
    for i in col:
        data_mean = df[i].mean()
        data_std = df[i].std()
        scaled = (df[i]-data_mean)/data_std
        df[i]=scaled
    return df

data_scaled(vip_data, col)

,CryoSleep,Cabin2,Cabin3,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,B,C,D,E,F,G,T,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
0,False,-1.167170,False,-0.953023,NaN,-0.302677,-0.030337,-0.220319,-0.286713,0.022392,...,0,0,0,0,1,0,0,1,0,0
1,False,-1.146095,False,-1.970321,NaN,-0.302677,-0.225656,-0.363966,-0.302602,-0.250833,...,0,0,0,0,0,1,0,0,0,1
2,True,-1.111608,True,0.403375,NaN,-0.302677,-0.225656,-0.363966,-0.302602,-0.250833,...,0,0,0,0,1,0,0,0,0,1
3,True,-1.092449,False,-1.020843,NaN,-0.302677,-0.225656,-0.363966,-0.302602,-0.250833,...,0,0,0,0,1,0,0,1,0,0
4,False,-1.136515,True,NaN,NaN,-0.247317,-0.165313,0.180697,-0.301808,1.115295,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,False,-0.550247,False,0.199915,NaN,-0.269203,0.452404,-0.363966,2.869616,0.076901,...,0,1,0,0,0,0,0,0,0,1
292,True,-0.619219,True,0.674654,NaN,-0.302677,-0.225656,-0.363966,-0.302602,-0.250833,...,0,1,0,0,0,0,0,0,0,1
293,False,NaN,True,0.606834,NaN,0.706693,-0.225656,-0.238275,-0.272413,-0.244020,...,0,0,0,0,0,0,0,0,0,1
294,False,2.432825,True,-0.546103,NaN,0.521298,-0.225656,-0.301120,-0.302602,-0.199050,...,0,0,0,0,1,0,0,0,0,1


In [ ]:
XX = vip_data.drop(['VIP'],axis=1)
yy = vip_data.VIP

In [ ]:
pred= xgb_model.predict(XX)
pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
XX.shape

(296, 23)

In [ ]:
vip_data

,CryoSleep,Cabin2,Cabin3,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,B,C,D,E,F,G,T,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
0,False,-1.167170,False,-0.953023,NaN,-0.302677,-0.030337,-0.220319,-0.286713,0.022392,...,0,0,0,0,1,0,0,1,0,0
1,False,-1.146095,False,-1.970321,NaN,-0.302677,-0.225656,-0.363966,-0.302602,-0.250833,...,0,0,0,0,0,1,0,0,0,1
2,True,-1.111608,True,0.403375,NaN,-0.302677,-0.225656,-0.363966,-0.302602,-0.250833,...,0,0,0,0,1,0,0,0,0,1
3,True,-1.092449,False,-1.020843,NaN,-0.302677,-0.225656,-0.363966,-0.302602,-0.250833,...,0,0,0,0,1,0,0,1,0,0
4,False,-1.136515,True,NaN,NaN,-0.247317,-0.165313,0.180697,-0.301808,1.115295,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,False,-0.550247,False,0.199915,NaN,-0.269203,0.452404,-0.363966,2.869616,0.076901,...,0,1,0,0,0,0,0,0,0,1
292,True,-0.619219,True,0.674654,NaN,-0.302677,-0.225656,-0.363966,-0.302602,-0.250833,...,0,1,0,0,0,0,0,0,0,1
293,False,NaN,True,0.606834,NaN,0.706693,-0.225656,-0.238275,-0.272413,-0.244020,...,0,0,0,0,0,0,0,0,0,1
294,False,2.432825,True,-0.546103,NaN,0.521298,-0.225656,-0.301120,-0.302602,-0.199050,...,0,0,0,0,1,0,0,0,0,1


In [ ]:
# 기존의 팀명 컬러 삭제
vip_data=vip_data.drop('VIP',axis=1)

KeyError: "['VIP'] not found in axis"

In [ ]:
vip_data.shape

(296, 23)

In [ ]:
pred.replace({0:False,1:True},inplace=True)

In [ ]:
pred = pd.DataFrame(pred)
vip_data.drop('answer',axis=1,inplace=True)
vip_data.insert(23,'answer',pred)
vip_data

,CryoSleep,Cabin2,Cabin3,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Earth,...,C,D,E,F,G,T,55 Cancri e,PSO J318.5-22,TRAPPIST-1e,answer
0,False,-1.167170,False,-0.953023,-0.302677,-0.030337,-0.220319,-0.286713,0.022392,1,...,0,0,0,1,0,0,1,0,0,False
1,False,-1.146095,False,-1.970321,-0.302677,-0.225656,-0.363966,-0.302602,-0.250833,1,...,0,0,0,0,1,0,0,0,1,False
2,True,-1.111608,True,0.403375,-0.302677,-0.225656,-0.363966,-0.302602,-0.250833,0,...,0,0,0,1,0,0,0,0,1,False
3,True,-1.092449,False,-1.020843,-0.302677,-0.225656,-0.363966,-0.302602,-0.250833,0,...,0,0,0,1,0,0,1,0,0,False
4,False,-1.136515,True,NaN,-0.247317,-0.165313,0.180697,-0.301808,1.115295,0,...,0,1,0,0,0,0,0,0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,False,-0.550247,False,0.199915,-0.269203,0.452404,-0.363966,2.869616,0.076901,0,...,1,0,0,0,0,0,0,0,1,False
292,True,-0.619219,True,0.674654,-0.302677,-0.225656,-0.363966,-0.302602,-0.250833,0,...,1,0,0,0,0,0,0,0,1,False
293,False,NaN,True,0.606834,0.706693,-0.225656,-0.238275,-0.272413,-0.244020,0,...,0,0,0,0,0,0,0,0,1,False
294,False,2.432825,True,-0.546103,0.521298,-0.225656,-0.301120,-0.302602,-0.199050,1,...,0,0,0,1,0,0,0,0,1,False


In [ ]:
vip_data.answer.value_counts()

False    278
True      18
Name: answer, dtype: int64

In [ ]:
vip_data.to_csv('vip결측값 채움.csv')